In [1]:
import argparse
from typing import Optional, Union

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from dataclasses import dataclass

import datasets
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import log_loss

from transformers import (
    AutoTokenizer,
    AutoConfig,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
    RobertaForMultipleChoice,
    AutoModelForSequenceClassification,
    LlamaModel,
    LlamaForSequenceClassification,
    BitsAndBytesConfig,
    get_polynomial_decay_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    TrainerCallback,
)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

from peft import (
    get_peft_config,
    PeftModel,
    PeftConfig,
    get_peft_model,
    LoraConfig,
    TaskType,
)
import os

In [2]:
from torch.utils.data import Dataset
class InstructionDataSet(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        super(InstructionDataSet, self).__init__()
        #self.data = data.sample(len(data), random_state=0).reset_index(drop=True)
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        # self.A_token = self.tokenizer.encode(text='A', add_special_tokens=False, truncation=True, )
        # self.B_token = self.tokenizer.encode(text='B', add_special_tokens=False, truncation=True, )
        # self.C_token = self.tokenizer.encode(text='C', add_special_tokens=False, truncation=True, )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        now_data = self.data.loc[index]
        idx = now_data['id']
        r_a = now_data['instruction_a']
        r_b = now_data['instruction_b']
        
        templete_part1 = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n\n"
        templete_part1_input_ids = self.tokenizer.encode(text=templete_part1, add_special_tokens=True,)
        
        model_a_input_ids = self.tokenizer.encode(text=r_a, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length // 2)
        model_b_input_ids = self.tokenizer.encode(text=r_b, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length // 2)
        templete_part2 = "###options\nA. Model A\nB. Model B\nC. Tie\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
        templete_part2_input_ids = self.tokenizer.encode(text=templete_part2, add_special_tokens=True,)
        
        input_ids = templete_part1_input_ids + model_a_input_ids + model_b_input_ids + templete_part2_input_ids

        return {
            "input_ids": input_ids,
            "id": idx
        }

In [3]:
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
@dataclass
class DataCollatorForInstruction:
    tokenizer: PreTrainedTokenizerBase
    model: Optional[Any] = None
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        labels = [feature["labels"] for feature in features] if "labels" in features[0].keys() else None
        # We have to pad the labels before calling `tokenizer.pad` as this method won't pad them and needs them of the
        # same length to return tensors.

        features = self.tokenizer.pad(
            features,
            padding=True,
            max_length=MAX_LENGTH,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors,
        )

        # prepare decoder_input_ids
        if (
                labels is not None
                and self.model is not None
                and hasattr(self.model, "prepare_decoder_input_ids_from_labels")
        ):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features["labels"])
            features["decoder_input_ids"] = decoder_input_ids
        # breakpoint() # [(len(features[i]['input_ids']),len(features[i]['labels'])) for i in range(4)]
        return features

In [4]:
test = pd.read_csv('dataset/random_instruction_valid.csv')#.sample(5).reset_index(drop = True)
test = test.loc[:500,:].reset_index(drop=True)
#sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

In [5]:
from tqdm import tqdm
def inference(model, test_dataloader):
    test_predictions = []
    for batch in tqdm(test_dataloader):
        for k in batch.keys():
            batch[k] = batch[k].to(device)
        with torch.no_grad():
            batch_input, idx = batch['input_ids'], batch['id']
            batch_input = {"input_ids": batch_input}
            outputs = model(**batch_input)
            #logits = outputs.logits.cpu().detach().numpy()
            response = model.generate(**batch_input, max_new_tokens=2, return_dict_in_generate=True, output_scores=True)
            #batch_input['input_ids'].shape[-1] + 1
            print(f"score {response.scores}")
            print(f"score is {response.scores[0]}")
            #print(f"response is {response}")
            #redict = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
            score = response.scores[0]
            A_prob, B_prob, C_prob = score[:,A_TOKEN_IDS], score[:,B_TOKEN_IDS], score[:,C_TOKEN_IDS]
            logits = torch.Tensor([[A_prob,B_prob,C_prob]])
            logits = torch.softmax(logits, dim=-1).cpu().numpy()
            print(f"logits is {logits}")
            node_result = [[idx[i],logits[i]] for i in range(batch_size)]

        test_predictions.append(node_result)
    return test_predictions

In [6]:
device = torch.device("cuda:0")

In [7]:
base_model = 'meta-llama/llama-3-transformers-8b-hf-v1'
model_path = "output/efficient-sponge-113/checkpoint-4"
MAX_LENGTH = 1500

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)
base_model_0 = AutoModelForCausalLM.from_pretrained(base_model,
                                                 config=config,
                                                 quantization_config=bnb_config,
                                                 torch_dtype=torch.bfloat16,
                                                 device_map="auto",
                                                 trust_remote_code=True)
base_model_0.config.pad_token_id = tokenizer.pad_token_id
base_model_0.resize_token_embeddings(len(tokenizer))
new_model = model_path
model0 = PeftModel.from_pretrained(base_model_0, new_model).to(device)
#model0 = model0.merge_and_unload()
model0.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128257, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [9]:

A_TOKEN_IDS = tokenizer.encode('A',add_special_tokens=True, truncation=True,
                                          max_length=1024)
B_TOKEN_IDS = tokenizer.encode('B',add_special_tokens=True, truncation=True,
                                          max_length=1024)
C_TOKEN_IDS = tokenizer.encode('C',add_special_tokens=True, truncation=True,
                                          max_length=1024)

In [10]:
tokenized_dataset = InstructionDataSet(test, tokenizer,MAX_LENGTH, 1)
# test_dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size = 2 ,shuffle=False, collate_fn=collate_fn)


data_collator = DataCollatorForInstruction(tokenizer=tokenizer)
batch_size = 1
test_dataloader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

In [11]:
sub_pred = inference(model = model0, test_dataloader = test_dataloader)

  0%|          | 0/501 [00:00<?, ?it/s]You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 1/501 [00:00<07:58,  1.04it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

score (tensor([[ 3.9062,  4.9688,  5.8750,  ..., -3.8750, -3.8750, -1.5234]],
       device='cuda:0'), tensor([[ 4.7500,  4.7188,  6.8125,  ...,  0.0806,  0.0806, -1.6172]],
       device='cuda:0'))
score is tensor([[ 3.9062,  4.9688,  5.8750,  ..., -3.8750, -3.8750, -1.5234]],
       device='cuda:0')
logits is [[0.3916697  0.50291383 0.10541646]]


  0%|          | 2/501 [00:01<05:15,  1.58it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


score (tensor([[ 5.1562,  5.8438,  6.7188,  ..., -4.5938, -4.5938, -2.4531]],
       device='cuda:0'), tensor([[ 4.5938,  5.9688,  7.1562,  ...,  3.0312,  3.0312, -2.4844]],
       device='cuda:0'))
score is tensor([[ 5.1562,  5.8438,  6.7188,  ..., -4.5938, -4.5938, -2.4531]],
       device='cuda:0')
logits is [[0.5511481  0.252334   0.19651791]]


  1%|          | 3/501 [00:01<04:11,  1.98it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

score (tensor([[ 4.0312,  5.0000,  6.5312,  ..., -3.2500, -3.2500, -0.8672]],
       device='cuda:0'), tensor([[ 3.7812,  5.3750,  7.1875,  ..., -1.9844, -1.9844, -0.7891]],
       device='cuda:0'))
score is tensor([[ 4.0312,  5.0000,  6.5312,  ..., -3.2500, -3.2500, -0.8672]],
       device='cuda:0')
logits is [[0.4213596  0.4083957  0.17024466]]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|█████████▉| 500/501 [03:32<00:00,  2.13it/s]

score (tensor([[ 5.5625,  6.3125,  7.3750,  ..., -4.8438, -4.8438, -0.0918]],
       device='cuda:0'), tensor([[ 5.0938,  6.3438,  7.7500,  ..., -3.0938, -3.0938, -0.2363]],
       device='cuda:0'))
score is tensor([[ 5.5625,  6.3125,  7.3750,  ..., -4.8438, -4.8438, -0.0918]],
       device='cuda:0')
logits is [[0.4345184  0.31790048 0.24758114]]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|██████████| 501/501 [03:32<00:00,  2.36it/s]

score (tensor([[ 4.0625,  5.0312,  6.3125,  ..., -4.1875, -4.1875,  0.3242]],
       device='cuda:0'), tensor([[ 4.2500,  5.7500,  7.6875,  ..., -0.7266, -0.7266, -0.3203]],
       device='cuda:0'))
score is tensor([[ 4.0625,  5.0312,  6.3125,  ..., -4.1875, -4.1875,  0.3242]],
       device='cuda:0')
logits is [[0.4172934  0.29590535 0.28680128]]


In [12]:
# 提取数据
processed_data = []
for item in sub_pred:
    item = item[0]
    id = item[0].item()  # 获取id
    array_values = item[1].tolist()  # 获取array并转换为列表
    processed_data.append([id] + array_values)

In [13]:
new_columns = ['id', 'winner_model_a', 'winner_model_b', 'winner_tie']
df = pd.DataFrame(processed_data, columns=new_columns)

In [14]:
df = df.groupby('id').mean().reset_index()

In [17]:
df

,id,winner_model_a,winner_model_b,winner_tie
0,30192,0.471409,0.377624,0.150967
1,1256092,0.568546,0.268562,0.162891
2,3258431,0.513956,0.387954,0.098090
3,4186011,0.325846,0.554282,0.119872
4,5717448,0.171352,0.721419,0.107229
...,...,...,...,...
400,599762723,0.337590,0.382539,0.279872
401,601396996,0.539887,0.275801,0.184312
402,602406101,0.421360,0.408396,0.170245
403,604482704,0.434518,0.317900,0.247581


In [18]:
str2num = {'A':0, "B":1, "C":2}

In [19]:
test['label_number'] = test.label.map(str2num)

In [20]:
from sklearn.metrics import log_loss

In [23]:
prediction = np.array(df[new_columns[1:]])

In [24]:
prediction

array([[0.4714089 , 0.37762392, 0.15096719],
       [0.56854641, 0.26856232, 0.16289128],
       [0.51395571, 0.38795412, 0.09809016],
       ...,
       [0.4213596 , 0.40839571, 0.17024466],
       [0.4345184 , 0.31790048, 0.24758114],
       [0.4172934 , 0.29590535, 0.28680128]])

In [29]:
test.drop_duplicates(on = ['id'])

TypeError: DataFrame.drop_duplicates() got an unexpected keyword argument 'on'

In [25]:
log_loss(test.label_number, prediction)

ValueError: Found input variables with inconsistent numbers of samples: [405, 501]

In [ ]:
3.176938522630114

In [ ]:
test['label'][:12]